<a href="https://colab.research.google.com/github/karjain/Anti_deepFake/blob/main/datapipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import argparse
from os.path import join
import cv2
import dlib
import glob
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
df = pd.read_json('/content/drive/MyDrive/Fake_dect/deepF/train_vid/metadata.json'
                  ,orient='index').reset_index()

df.head()

,index,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [ ]:
import glob

video_files_path =  glob.glob('/content/drive/MyDrive/Fake_dect/deepF/train_vid/*.mp4')
no_files=len(video_files_path)
processed=0
for video_file in video_files_path:
  
  reader = cv2.VideoCapture(video_file)
  video_fn = video_file.split('/')[-1].split('.')[0]
  video_name=video_fn+".mp4"
  label = df.loc[df['index']==video_name,'label'].item()

  #print(video_fn)

  i=0
  #os.makedirs(output_path, exist_ok=True)

  fps = reader.get(cv2.CAP_PROP_FPS)
  num_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
  #print("num of frames: ",num_frames)
 
  # Face detector
  face_detector = dlib.get_frontal_face_detector()
 
  # Text variables
  #font_face = cv2.FONT_HERSHEY_SIMPLEX
 
  # Frame numbers and length of output video
  frame_num = 0
  c=1
  while reader.isOpened():
      _, image = reader.read()
      if image is None:
          break
      if c%33==0:
        frame_num += 1
  
        # Image size
        height, width = image.shape[:2]
  
  
        # . Detect with dlib
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_detector(gray, 1)
        if len(faces):
            # For now only take biggest face
            for face in faces:
              # Face crop with dlib and bounding box scale enlargement
              x, y, size = get_boundingbox(face, width, height)
              cropped_face = image[y:y+size, x:x+size]
              if( label=='FAKE' ):
                cv2.imwrite("/content/drive/MyDrive/Fake_dect/deepF/train_img/fake/"+video_fn+str(i)+'.jpg',cropped_face)
              else:
                cv2.imwrite("/content/drive/MyDrive/Fake_dect/deepF/train_img/real/"+video_fn+str(i)+'.jpg',cropped_face)
              
              i+=1
              #cv2.waitKey(33)
        
      c+=1
  print(video_name, " ", label)
  print("total frames taken: ",frame_num)
  processed+=1
  print("Progress= ", (processed/no_files)*100  )
      
  

###Functions


In [ ]:
def get_boundingbox(face, width, height, scale=1.3, minsize=None):
    """
    Expects a dlib face to generate a quadratic bounding box.
    :param face: dlib face class
    :param width: frame width
    :param height: frame height
    :param scale: bounding box size multiplier to get a bigger face region
    :param minsize: set minimum bounding box size
    :return: x, y, bounding_box_size in opencv form
    """
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * scale)
    if minsize:
        if size_bb < minsize:
            size_bb = minsize
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

    # Check for out of bounds, x-y top left corner
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    # Check for too big bb size for given x, y
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)

    return x1, y1, size_bb

In [ ]:
def video_to_croped_frames(video_files_root, out_path, metafile_path, frame_rate=33):
  file_extensions = ['mp4','avi']
  #file_extension eg 'mp4', 'avi' 
  
  df = pd.read_json(metafile_path, orient='index')                
  video_files_path=[]
  for ext in file_extensions:
    video_files_path.extend(glob.glob(video_files_root + '/*.'+ext))
    print(type(video_files_path))
  no_files = len(video_files_path)

  os.makedirs(output_path+'/fake', exist_ok=True)
  os.makedirs(output_path+'/real', exist_ok=True)
  
  videos_processed=0

  for video_file in video_files_path:
    reader = cv2.VideoCapture(video_file)
    
    video_name = video_file.split('/')[-1]
    video_fn = video_name.split('.')[0]

    label = df.loc[video_name,'label']

    #fps = reader.get(cv2.CAP_PROP_FPS)
    #num_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
    #print("num of frames: ",num_frames)
  
    # Face detector
    face_detector = dlib.get_frontal_face_detector()
    
    # Frame numbers and length of output video
    i=0
    c=1
    while reader.isOpened():
        _, image = reader.read()
        if image is None:
            break
        if c%frame_rate==0:
    
          # Image size
          height, width = image.shape[:2]

          # . Detect with dlib
          gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          faces = face_detector(gray, 1)
          if len(faces):
              for face in faces:
                # Face crop with dlib and bounding box scale enlargement
                x, y, size = get_boundingbox(face, width, height)
                cropped_face = image[y:y+size, x:x+size]
                if( label=='FAKE' ):
                  cv2.imwrite(out_path + "/fake/" + video_fn+str(i) + '.jpg', cropped_face)
                else:
                  cv2.imwrite(out_path + "/real/" + video_fn+str(i) + '.jpg', cropped_face)
                
                i+=1
                #cv2.waitKey(33)
          
        c+=1
    print(video_name, " ", label)
    print("total frames written: ",i)
    videos_processed+=1
    print("Progress= ", (videos_processed/no_files)*100  )
      
  

In [ ]:
video_to_croped_frames('/content/drive/MyDrive/Fake_dect/deepF/train_vid', '/content/drive/MyDrive/Fake_dect/deepF', '/content/drive/MyDrive/Fake_dect/deepF/train_vid/metadata.json', frame_rate=33)

<class 'list'>
<class 'list'>
aagfhgtpmv.mp4   FAKE
total frames written:  9
Progress=  0.25
aapnvogymq.mp4   FAKE
total frames written:  14
Progress=  0.5


In [ ]:
#!rm /content/drive/MyDrive/Fake_dect/deepF/train_img/fake/*
#!rm /content/drive/MyDrive/Fake_dect/deepF/train_img/real/*


###**Fake** rename

In [ ]:
import os
x=1 
root = "/content/drive/MyDrive/Fake_dect/trail"
for count, filename in enumerate(os.listdir(root)): 
        print(filename)

        dst ="/face" + str(x) + ".jpg"
        src =root+ '/'+filename 
        
        dst ='/content/drive/MyDrive/Fake_dect/renamed/face' + str(x) + '.jpg'
        os.rename(src, dst)
        x+=1 
  

###Real rename

In [ ]:
count=0
for x in range(1,20000):
  src = '/content/drive/MyDrive/Fake_dect/train_img/Real/' + str(x) + '.jpg'
  if(os.path.exists(src)):
    #dst = '/content/drive/MyDrive/Fake_dect/train_img/Real_renamed/' + 'face' + str(x) + '.jpg'
    #os.rename(src, dst)
    count+=1
    print(count)
print("count",count)

In [ ]:
!rm '/content/drive/MyDrive/Fake_dect/train_img/Real_renamed/'